### Structure of DataFrames
* **Provences**: [Provence, Domain, Regent, Terrain, Loyalty, Taxation, Population, Magic, Castle, Capital, Position, Troops]
* **Holdings**: [Provence, Domain, Regent, Type, Level]
* **Regents**: [Regent, Full Name, Player, Class, Level, Alignment, Str, Dex, Con, Int, Wis, Cha, Insight, Deception, Persuasion, Regency Points, Gold Bars, Regency Bonus, Attitude, Lieutenants]
* **Geography**: [Provence, Neighbor, Border, Road, Caravan, Shipping]
* **Relationship**: [Regent, Other, Diplomacy, Payment, Vassalage]
* **Troops**: [Regent, Provence, Type, Cost, CR]

* **Seasons**: A dctionary of season-dataframes (to keep track of waht happened)



In [1]:
from regency import Regency

Game = Regency()
Game.random_events()
Game.collect_regency_points()
Game.domain_initiative()
Game.collect_gold_bars()
Game.maintenance_costs()

Game.Seasons[0]

,Regent,Player,Random Event,Regency Points,Initiative,Gold Bars,Revenue,Cost,Court
0,EC,True,No Event,35,21,5,14,9,Average
1,LPA,False,No Event,14,20,4,11,7,Average
2,TBM,False,No Event,12,18,0,0,0,Dormant
3,WM,False,No Event,8,18,1,2,1,Dormant
4,IHH,False,No Event,7,17,2,4,2,Bare
5,DW,False,Brigandage,23,16,11,18,7,Average
6,Sw,False,Corruption/Crime,12,16,0,0,0,Dormant
7,TCV,False,Natural Event,2,15,0,1,1,Dormant
8,HA,False,No Event,4,12,1,3,2,Bare
9,JR,False,No Event,36,12,4,13,9,Average


In [2]:
Game.Holdings[Game.Holdings['Regent'] == 'TCV']

,Provence,Regent,Type,Level
52,Moriel,TCV,Temple,1


In [3]:
temp = Game.Regents[Game.Regents['Lieutenants'].str.len()>0]

In [4]:
for i, row in temp.iterrows():
    print(len(row['Lieutenants']))

1
1
